# 📊 Customer Profitability Modeling

## 🧠 Objective

The goal of this notebook is to estimate and analyze customer profitability for an AI infrastructure company like Cohere.

We will:

- Segment customers into cohorts based on observable traits (industry, size, geography, etc.)
- Estimate **Customer Lifetime Value (CLTV)** per cohort
- Estimate **Customer Acquisition Cost (CAC)** per cohort
- Analyze **CLTV:CAC ratio** to determine strategic efficiency
- Recommend which cohorts to target or deprioritize

This analysis supports strategic decisions around:
- Sales & marketing focus
- Product prioritization
- Go-to-market efficiency


In [ ]:
## 🔧 Section 2: Data Setup & Imports

To perform our customer profitability analysis, we'll start by importing standard Python libraries and creating a mock dataset to simulate different customer segments and financial metrics.

We'll model:
- Customer segments (e.g., startup, enterprise, gov)
- Revenue per customer over time
- Gross margin assumptions
- Acquisition cost estimates


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# For consistent visuals
sns.set(style="whitegrid")

# Random seed for reproducibility
np.random.seed(42)

# Simulate 150 customers across 3 segments
n_customers = 150
segments = ['Startup', 'Enterprise', 'Government']

data = {
    'Customer_ID': range(1, n_customers + 1),
    'Segment': np.random.choice(segments, size=n_customers, p=[0.5, 0.3, 0.2]),
    'Annual_Revenue': np.random.normal(loc=100000, scale=30000, size=n_customers).round(2),
    'Gross_Margin': np.random.uniform(0.6, 0.85, size=n_customers).round(2),
    'CAC': np.random.normal(loc=20000, scale=5000, size=n_customers).round(2),
    'Retention_Years': np.random.randint(1, 6, size=n_customers)  # Simulate 1 to 5 years
}

df = pd.DataFrame(data)

# Clean any negative or weird values
df['Annual_Revenue'] = df['Annual_Revenue'].clip(lower=20000)
df['CAC'] = df['CAC'].clip(lower=5000)

# Preview the dataset
df.head()


In [ ]:
## 🔍 Section 3: Cohort Segmentation & Visualization

To understand how different types of customers perform, we’ll segment them into cohorts based on their profile — in this case, by the `Segment` variable (Startup, Enterprise, Government).

We'll visualize differences across cohorts in:

- Annual revenue
- Gross margin
- CAC
- Retention

This will help us detect early signs of which customer types are most profitable or strategic.


In [ ]:
# Summary statistics by segment
cohort_summary = df.groupby('Segment').agg({
    'Annual_Revenue': 'mean',
    'Gross_Margin': 'mean',
    'CAC': 'mean',
    'Retention_Years': 'mean'
}).round(2).reset_index()

cohort_summary


In [ ]:
fig, axs = plt.subplots(2, 2, figsize=(14, 10))

sns.barplot(x='Segment', y='Annual_Revenue', data=df, ax=axs[0, 0])
axs[0, 0].set_title('Avg Annual Revenue by Segment')

sns.barplot(x='Segment', y='Gross_Margin', data=df, ax=axs[0, 1])
axs[0, 1].set_title('Avg Gross Margin by Segment')

sns.barplot(x='Segment', y='CAC', data=df, ax=axs[1, 0])
axs[1, 0].set_title('Avg CAC by Segment')

sns.barplot(x='Segment', y='Retention_Years', data=df, ax=axs[1, 1])
axs[1, 1].set_title('Avg Retention (Years) by Segment')

plt.tight_layout()
plt.show()


In [ ]:
## 💰 Section 4: CLTV Estimation

To estimate Customer Lifetime Value (CLTV), we’ll use the following formula:

**CLTV = (Annual Revenue × Gross Margin %) × Retention Years**

This represents the gross profit generated over the lifetime of the customer. It allows us to compare long-term value across different segments.


In [ ]:
# Compute CLTV per customer
df['CLTV'] = (df['Annual_Revenue'] * df['Gross_Margin']) * df['Retention_Years']
df['CLTV'] = df['CLTV'].round(2)

# Show a few rows
df[['Customer_ID', 'Segment', 'Annual_Revenue', 'Gross_Margin', 'Retention_Years', 'CLTV']].head()


In [ ]:
# Mean CLTV by segment
cltv_summary = df.groupby('Segment')['CLTV'].mean().round(2).reset_index()

# Plot
plt.figure(figsize=(8, 5))
sns.barplot(x='Segment', y='CLTV', data=cltv_summary)
plt.title('Average CLTV by Segment')
plt.ylabel('Customer Lifetime Value ($)')
plt.show()


In [ ]:
## ⚖️ Section 5: CAC & CLTV:CAC Ratio Analysis

To assess the efficiency of Cohere's customer acquisition strategy, we calculate the **CLTV:CAC ratio**.

- A ratio > 3.0 is typically considered strong
- A ratio < 1.0 suggests unsustainable customer acquisition

We'll compute this ratio per customer and evaluate it across cohorts to identify which segments are most efficient to acquire.


In [ ]:
# Compute CLTV:CAC ratio
df['CLTV_CAC_Ratio'] = (df['CLTV'] / df['CAC']).round(2)

# View top 5 customers
df[['Customer_ID', 'Segment', 'CLTV', 'CAC', 'CLTV_CAC_Ratio']].head()


In [ ]:
plt.figure(figsize=(8, 5))
sns.barplot(x='Segment', y='CLTV_CAC_Ratio', data=df)
plt.title('Avg CLTV:CAC Ratio by Segment')
plt.ylabel('CLTV to CAC Ratio')
plt.show()
